In [1]:
import pandas as pd
import os
import datetime
import holidays
import csv
import io
from dataclasses import dataclass, asdict

In [936]:
from dataclasses import dataclass

@dataclass
class HDPowerData:
    power_plant: str = ""
    generation: float = 0
    residual_power: int = 0
    time_slot: str = 0

@dataclass
class HDClientData:
    client_id: str
    total_utility_power: float = 0  # 總市電
    total_quota: float = 0  # 總綠電轉供 (stage1 + stage2)
    time_slot: str = 0

@dataclass
class PowerData02:
    power_plant: str = ""
    generation: float = 0
    residual_power: int = 0
    time_slot: str = 0

@dataclass
class ClientData02:
    client_id: str
    total_utility_power: float = 0  # 總市電
    total_quota: float = 0  # 總綠電轉供 (stage1 + stage2)
    time_slot: str = 0



In [1197]:
def __load_power_plant_dicts(filepath):
    power_plant_dicts = {}
    with open(filepath + os.path.sep + 'plant_information.csv', 'r', encoding="utf-8-sig") as f:
        rows = csv.reader(f, delimiter=',')
        _headers = next(rows)
        for row in rows:
            power_plant_dicts[row[0]] = {}
            # HDRE asks to round FIT to 4 digits
            #power_plant_dicts[row[0]]["surplus_price"] = float(row[1])
            power_plant_dicts[row[0]]["surplus_price"] = round(float(row[1]), 4)

            # HDRE asks to round capacity to 2 digits
            #power_plant_dicts[row[0]]["capacity"] = float(row[2])
            power_plant_dicts[row[0]]["capacity"] = round(float(row[2]), 2)
            power_plant_dicts[row[0]]["power_plant_id"] = row[3]
            power_plant_dicts[row[0]]["surplus_power"] = 0
    return power_plant_dicts

def __load_power_client_dicts(filepath):
    power_plant_dicts = {}
    with open(filepath + os.path.sep + 'client_information.csv', 'r', encoding="utf-8-sig") as f:
        rows = csv.reader(f, delimiter=',')
        _headers = next(rows)
        for row in rows:
            power_plant_dicts[row[1]] = {}
            power_plant_dicts[row[1]]["綠電售價"] = round(float(row[6]), 4)
    return power_plant_dicts

@dataclass
class HDPlant:
    plant_name: str
    total_transfered: float = 0  # 總轉供
    residual_power: int = 0
    time_slot: str = 0


class HDPlantRecorder:
    def __init__(self, plant_names):
        self.hd_plant_dict = {name: HDPlant(name) for name in plant_names}

    def add_power_transfered(self, plant_name, power_transfered, time_slot, residual_power):
        obj = self.hd_plant_dict[plant_name]
        obj.total_transfered = float(power_transfered)
        obj.time_slot = time_slot
        obj.residual_power = residual_power

    def get_power_transfered(self, plant_name):
        return self.hd_plant_dict[plant_name].total_transfered

    def get_residual_power(self, plant_name):
        return self.hd_plant_dict[plant_name].residual_power

    def get_plant_name(self, plant_name):
        return self.hd_plant_dict[plant_name].plant_name

    def summarize(self):
        for name, obj in self.hd_plant_dict.items():
            print(asdict(obj))

@dataclass
class HDPlant02:
    plant_name: str
    total_transfered: float = 0  # 總轉供
    residual_power: int = 0
    time_slot: str = 0


class HDPlantRecorder02:
    def __init__(self, plant_names):
        self.hd_plant_dict = {name: HDPlant(name) for name in plant_names}

    def add_power_transfered(self, plant_name, power_transfered, residual_power, time_slot):
        obj = self.hd_plant_dict[plant_name]
        obj.total_transfered = float(power_transfered)
        obj.residual_power = residual_power
        obj.time_slot = time_slot

    def get_power_transfered(self, plant_name):
        return self.hd_plant_dict[plant_name].total_transfered

    def get_residual_power(self, plant_name):
        return self.hd_plant_dict[plant_name].residual_power

    def get_plant_name(self, plant_name):
        return self.hd_plant_dict[plant_name].plant_name

    def summarize(self):
        for name, obj in self.hd_plant_dict.items():
            print(asdict(obj))


@dataclass
class HDClient:
    client_id: str
    total_utility_power: float = 0  # 總市電
    total_quota: float = 0  # 總綠電轉供 (stage1 + stage2)
    time_slot: str = 0


class HDClientRecorder:
    def __init__(self, client_ids):
        self.hd_client_dict = {cid: HDClient(cid) for cid in client_ids}

    def add_quota(self, client_id, quota):
        obj = self.hd_client_dict[client_id]
        obj.total_quota = quota

    def get_total_consumption(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_power_consumption

    def get_total_quota(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_quota

    def get_total_utility(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_utility_power

    def get_hd_client(self, client_id):
        return self.hd_client_dict[client_id].client_id

    def add_power_consumption(self, client_id, power_consumption):
        obj = self.hd_client_dict[client_id]
        obj.total_power_consumption += power_consumption

    def add_utility_power(self, client_id, utility_power):
        obj = self.hd_client_dict[client_id]
        obj.total_utility_power += utility_power

    def add_time_slot(self, client_id, time_slot):
        obj = self.hd_client_dict[client_id]
        obj.time_slot = time_slot

    def summarize(self):
        for _name, obj in self.hd_client_dict.items():
            print(asdict(obj))

@dataclass
class HDClient02:
    client_id: str
    total_utility_power: float = 0  # 總市電
    total_quota: float = 0  # 總綠電轉供 (stage1 + stage2)
    time_slot: str = 0


class HDClientRecorder02:
    def __init__(self, client_ids):
        self.hd_client_dict = {cid: HDClient(cid) for cid in client_ids}

    def add_quota(self, client_id, quota):
        obj = self.hd_client_dict[client_id]
        obj.total_quota = quota

    def get_total_consumption(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_power_consumption

    def get_total_quota(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_quota

    def get_total_utility(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_utility_power

    def get_hd_client(self, client_id):
        return self.hd_client_dict[client_id].client_id

    def add_power_consumption(self, client_id, power_consumption):
        obj = self.hd_client_dict[client_id]
        obj.total_power_consumption += power_consumption

    def add_utility_power(self, client_id, utility_power):
        obj = self.hd_client_dict[client_id]
        obj.total_utility_power = utility_power

    def add_time_slot(self, client_id, time_slot):
        obj = self.hd_client_dict[client_id]
        obj.time_slot = time_slot

    def summarize(self):
        for _name, obj in self.hd_client_dict.items():
            print(asdict(obj))


def read_predefined_contracts(csv_fn):
    assert csv_fn.endswith(".csv"), f"Wrong csv_fn: {csv_fn}"
    power_plant_dict = {}
    with io.open(csv_fn, encoding="utf-8-sig") as _fp:
        reader = csv.reader(_fp, delimiter=",")
        rows = [row for row in reader]
    plant_name = rows[0][1]
    plant_generation = rows[0][2]
    docs = []
    for row in rows[1:]:
        doc = {
            'type': row[0],
            plant_name: row[1],
            plant_generation: row[2]
        }
        docs.append(doc)

    return docs

def __update_plant_recorder(plant_recorder, obj):
    for i in obj:
        plant_name = i.power_plant
        quota_sum = i.generation
        time_slot = i.time_slot
        residual_power = i.residual_power
        plant_recorder.add_power_transfered(plant_name, quota_sum, residual_power, time_slot)

def __update_client_recorder(client_record, obj):
    for i in obj:
        client_id = i.client_id
        total_utility_power = i.total_utility_power
        time_slot = i.time_slot
        total_quota = i.total_quota
        client_record.add_utility_power(client_id, total_utility_power)
        client_record.add_time_slot(client_id, time_slot)
        client_record.add_quota(client_id, total_quota)

In [1198]:
plant_ret = read_predefined_contracts('power_generation.csv')
power_plant_dict = __load_power_plant_dicts('./contract_3_5_month_minutes/')
print(power_plant_dict)
HDPlantRecorder(power_plant_dict.keys()).summarize()
power_client_dict = __load_power_client_dicts('./contract_3_5_month_minutes/')
print(power_client_dict)
client_record = HDClientRecorder(power_client_dict.keys())
client_record.summarize()

{'電廠001': {'surplus_price': 5.0004, 'capacity': 9000.0, 'power_plant_id': '20655320001', 'surplus_power': 0}, '電廠002': {'surplus_price': 4.9552, 'capacity': 12000.0, 'power_plant_id': '20655320002', 'surplus_power': 0}}
{'plant_name': '電廠001', 'total_transfered': 0, 'residual_power': 0, 'time_slot': 0}
{'plant_name': '電廠002', 'total_transfered': 0, 'residual_power': 0, 'time_slot': 0}
{'64-67-0001-13-1': {'綠電售價': 5.5376}, '64-67-0002-13-1': {'綠電售價': 6.2255}, '64-67-0003-13-1': {'綠電售價': 5.9575}, '64-67-0004-13-1': {'綠電售價': 6.0537}, '64-67-0005-13-1': {'綠電售價': 5.5821}}
{'client_id': '64-67-0001-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0002-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0003-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0004-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0005-13-1', 'total_utility_power'

In [1199]:
df = pd.read_csv('./contract_3_5_month_minutes/' + 'client_information.csv')
df = df.sort_values(by='綠電售價', ascending=False)
print(df['用電端電號'])
df_t = pd.read_csv('power_consumption_test.csv')
df_t = df_t.to_dict('index')
df_t

1    64-67-0002-13-1
3    64-67-0004-13-1
2    64-67-0003-13-1
4    64-67-0005-13-1
0    64-67-0001-13-1
Name: 用電端電號, dtype: object


{0: {'type': 0,
  '64-67-0001-13-1': 379638,
  '64-67-0002-13-1': 455565.6,
  '64-67-0003-13-1': 303710.4,
  '64-67-0004-13-1': 560350,
  '64-67-0005-13-1': 2186722.5},
 1: {'type': 2,
  '64-67-0001-13-1': 180670,
  '64-67-0002-13-1': 16804.0,
  '64-67-0003-13-1': 144536.0,
  '64-67-0004-13-1': 481250,
  '64-67-0005-13-1': 1040683.5}}

In [1200]:
power_plant_dict = __load_power_plant_dicts('./')
power_plant_dict
plant_record = HDPlantRecorder(power_plant_dict.keys())
plant_record.summarize()

{'plant_name': '電廠001', 'total_transfered': 0, 'residual_power': 0, 'time_slot': 0}
{'plant_name': '電廠002', 'total_transfered': 0, 'residual_power': 0, 'time_slot': 0}


In [1201]:
generation_list = []
generation_list2 = []
for doc in plant_ret:
    if doc['type'] == '0':
        ret = {
            "power_plant": doc["power_plant"],
            "generation": doc['power_generation']
        }
        generation_list.append(ret)
        # print(power_plant, generation)
    elif doc['type'] == '2':
        ret = {
            "power_plant": doc["power_plant"],
            "generation": doc['power_generation']
        }
        generation_list2.append(ret)


def cal_generation_0(power_list, con_list, c_list):
    rm_list = []
    for i in power_list:
        if i['power_plant'] == "電廠002":
            res = float(i.get("generation"))
            for j in c_list:
                print(j)
                res = res - float(con_list.get(0).get(j))
                docs = [
                    HDClientData(
                        client_id= j,
                        total_quota= float(con_list.get(0).get(j)),
                        total_utility_power= 0,
                        time_slot= '0'
                    )
                ]
                __update_client_recorder(client_record, docs)
                rm_list.append(j)
                if res < 0:
                    doc = [
                        HDPowerData(
                            power_plant=i['power_plant'],
                            generation=i['generation'],
                            time_slot= '0',
                            residual_power = 0
                        )
                    ]
                    __update_plant_recorder(plant_record, doc)

                    docs = [
                        HDClientData(
                            client_id= j,
                            total_quota=  res - float(con_list.get(0).get(j)),
                            total_utility_power= 0,
                            time_slot= '0'
                        )
                    ]
                    __update_client_recorder(client_record, docs)
                    print("no power")
                    break
    new_df_list = c_list[c_list.isin(rm_list) == False]
    return res, new_df_list

def cal_stage_2(power_list, power, con_list, new_df_list):
    for i in power_list:
        if i['power_plant'] == "電廠001":
            for j in new_df_list:
                print(j)
                res = float(i.get("generation"))
                res = res + float(power)
                doc = [
                    HDPowerData(
                        power_plant=i['power_plant'],
                        generation=i['generation'],
                        time_slot= '0',
                        residual_power = 0
                    )
                ]
                __update_plant_recorder(plant_record, doc)
                docs = [
                        HDClientData(
                            client_id= j,
                            total_quota=  '0',
                            total_utility_power= con_list.get(0).get(j),
                            time_slot= '0'
                        )
                    ]
                __update_client_recorder(client_record, docs)
            if res < 0:
                print(abs(con_list.get(0).get('64-67-0004-13-1') + res))
                docs = [
                        HDClientData(
                            client_id= '64-67-0004-13-1',
                            total_quota=  con_list.get(0).get('64-67-0004-13-1') + res,
                            total_utility_power= abs(res),
                            time_slot= '0'
                        )
                    ]
                __update_client_recorder(client_record, docs)


# def cal_stage_3(power_list, power, con_list, )



In [1202]:
res, c_list = cal_generation_0(generation_list, df_t, df['用電端電號'])
print(c_list)

64-67-0002-13-1
64-67-0004-13-1
no power
2    64-67-0003-13-1
4    64-67-0005-13-1
0    64-67-0001-13-1
Name: 用電端電號, dtype: object


In [1203]:
cal_stage_2(generation_list, res, df_t, c_list)

64-67-0003-13-1
64-67-0005-13-1
64-67-0001-13-1
550634.4


In [1204]:
plant_record.summarize()

{'plant_name': '電廠001', 'total_transfered': 447200.0, 'residual_power': '0', 'time_slot': 0}
{'plant_name': '電廠002', 'total_transfered': 559000.0, 'residual_power': '0', 'time_slot': 0}


In [1205]:
client_record.summarize()

{'client_id': '64-67-0001-13-1', 'total_utility_power': 379638, 'total_quota': '0', 'time_slot': '0'}
{'client_id': '64-67-0002-13-1', 'total_utility_power': 0, 'total_quota': 455565.6, 'time_slot': '0'}
{'client_id': '64-67-0003-13-1', 'total_utility_power': 303710.4, 'total_quota': '0', 'time_slot': '0'}
{'client_id': '64-67-0004-13-1', 'total_utility_power': 9715.599999999977, 'total_quota': 550634.4, 'time_slot': '0'}
{'client_id': '64-67-0005-13-1', 'total_utility_power': 2186722.5, 'total_quota': '0', 'time_slot': '0'}


In [1180]:
def cal_generation_2(power_list, con_list, c_list):
    rm_list = []
    for i in power_list:
        if i['power_plant'] == "電廠002":
            res = float(i.get("generation"))
            for j in c_list:
                res = res - float(con_list.get(1).get(j))
                print(res)
                docs = [
                    HDClient02(
                        client_id= j,
                        total_quota= float(con_list.get(1).get(j)),
                        total_utility_power= 0,
                        time_slot= '2'
                    )
                ]
                __update_client_recorder(client_record02, docs)
                rm_list.append(j)
                if res < 0:
                    doc = [
                        PowerData02(
                            power_plant=i['power_plant'],
                            generation=i['generation'],
                            residual_power = 0,
                            time_slot= '2'
                        )
                    ]
                    __update_plant_recorder(plant_record02, doc)

                    docs = [
                        HDClient02(
                            client_id= j,
                            total_quota=  res - float(con_list.get(1).get(j)),
                            total_utility_power= 0,
                            time_slot= '2'
                        )
                    ]
                    __update_client_recorder(client_record02, docs)
                    print("no power")
                    break
    new_df_list = c_list[c_list.isin(rm_list) == False]
    return res, new_df_list

def cal_stage_2_2(power_list, power, con_list, new_df_list):
    rm_list = []
    for i in power_list:
        if i['power_plant'] == "電廠001":
            for j in new_df_list:
                print(j)
                ds = float(i.get("generation"))
                ds = ds + float(power)
                ft = ds - con_list.get(1).get(j)
                doc = [
                    PowerData02(
                        power_plant=i['power_plant'],
                        generation=i['generation'],
                        residual_power = 0,
                        time_slot= '2'
                    )
                ]
                __update_plant_recorder(plant_record02, doc)

                docs = [
                        HDClient02(
                            client_id= j,
                            total_quota=  con_list.get(1).get(j),
                            total_utility_power= '0',
                            time_slot= '2'
                        )
                    ]
                __update_client_recorder(client_record02, docs)
                rm_list.append(j)
                if float(power) < 0:
                    print("補power")
                    docs = [
                            HDClient02(
                                client_id= '64-67-0004-13-1',
                                total_quota=  con_list.get(1).get('64-67-0004-13-1'),
                                total_utility_power= '0',
                                time_slot= '2'
                            )
                        ]
                    __update_client_recorder(client_record02, docs)
                    break
    print(rm_list)
    new_df_list = c_list[c_list.isin(rm_list) == False]
    for x in new_df_list:
        tt = ft - con_list.get(1).get(x)
        print(x)
        rm_list.append(x)
        if float(tt) < 0:
            docs = [
                HDClient02(
                    client_id= x,
                    total_quota=  ft,
                    total_utility_power= abs(tt),
                    time_slot= '2'
                )
            ]
            __update_client_recorder(client_record02, docs)
            break
    print(rm_list)

    new_df_list = c_list[c_list.isin(rm_list) == False]
    for z in new_df_list:
        # print(z, con_list.get(1).get(z))
        docs = [
            HDClient02(
                client_id= z,
                total_quota=  '0',
                total_utility_power= con_list.get(1).get(z),
                time_slot= '2'
            )
        ]
        __update_client_recorder(client_record02, docs)
        break
                # else:
                #     docs = [
                #         HDClient02(
                #             client_id= j,
                #             total_quota=  '0',
                #             total_utility_power= con_list.get(0).get(j),
                #             time_slot= '2'
                #         )
                #     ]
                # __update_client_recorder(client_record02, docs)

                # docs = [
                #         HDClientData(
                #             client_id= j,
                #             total_quota=  '0',
                #             total_utility_power= con_list.get(2).get(j),
                #             time_slot= '0'
                #         )
                #     ]
                # __update_client_recorder(client_record, docs)

In [1181]:
power_plant_dict = __load_power_plant_dicts('./')
power_plant_dict
plant_record02 = HDPlantRecorder02(power_plant_dict.keys())

power_client_dict = __load_power_client_dicts('./contract_3_5_month_minutes/')
power_client_dict
client_record02 = HDClientRecorder02(power_client_dict.keys())
client_record02.summarize()

{'client_id': '64-67-0001-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0002-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0003-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0004-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0005-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}


In [1182]:
res, c_list = cal_generation_2(generation_list2, df_t, df['用電端電號'])
cal_stage_2_2(generation_list2, res, df_t, c_list)

411546.0
-69704.0
no power
64-67-0003-13-1
補power
['64-67-0003-13-1']
64-67-0005-13-1
['64-67-0003-13-1', '64-67-0005-13-1']


In [1183]:
plant_record02.summarize()

{'plant_name': '電廠001', 'total_transfered': 342680.0, 'residual_power': 0, 'time_slot': '2'}
{'plant_name': '電廠002', 'total_transfered': 428350.0, 'residual_power': 0, 'time_slot': '2'}


In [1184]:
client_record02.summarize()

{'client_id': '64-67-0001-13-1', 'total_utility_power': 180670, 'total_quota': '0', 'time_slot': '2'}
{'client_id': '64-67-0002-13-1', 'total_utility_power': 0, 'total_quota': 16804.0, 'time_slot': '2'}
{'client_id': '64-67-0003-13-1', 'total_utility_power': '0', 'total_quota': 144536.0, 'time_slot': '2'}
{'client_id': '64-67-0004-13-1', 'total_utility_power': '0', 'total_quota': 481250, 'time_slot': '2'}
{'client_id': '64-67-0005-13-1', 'total_utility_power': 912243.5, 'total_quota': 128440.0, 'time_slot': '2'}


In [1185]:
plant_record.summarize()

{'plant_name': '電廠001', 'total_transfered': 447200.0, 'residual_power': '0', 'time_slot': 0}
{'plant_name': '電廠002', 'total_transfered': 559000.0, 'residual_power': '0', 'time_slot': 0}


In [1186]:
client_record.summarize()

{'client_id': '64-67-0001-13-1', 'total_utility_power': 379638, 'total_quota': '0', 'time_slot': '0'}
{'client_id': '64-67-0002-13-1', 'total_utility_power': 0, 'total_quota': 455565.6, 'time_slot': '0'}
{'client_id': '64-67-0003-13-1', 'total_utility_power': 303710.4, 'total_quota': '0', 'time_slot': '0'}
{'client_id': '64-67-0004-13-1', 'total_utility_power': 9715.599999999977, 'total_quota': 550634.4, 'time_slot': '0'}
{'client_id': '64-67-0005-13-1', 'total_utility_power': 2186722.5, 'total_quota': '0', 'time_slot': '0'}


In [1191]:
power_client_dict = __load_power_client_dicts('./contract_3_5_month_minutes/')
total_quota = 0
total_utility_power = 0
total_profit = 0

total_quota_2 = 0
total_utility_power_2 = 0
total_profit_2 = 0

for i in power_client_dict:
    profit = round(float(client_record.get_total_quota(i)) * float(power_client_dict.get(i).get('綠電售價')), 0)
    total_quota += float(client_record.get_total_quota(i))
    total_utility_power += float(client_record.get_total_utility(i))
    total_profit += profit
    print(f"電號: {client_record.get_hd_client(i)} 轉供: {float(client_record.get_total_quota(i))} 市電: {float(client_record.get_total_utility(i))} 利潤: {profit} 時段: 0")
print(f"總轉供: {total_quota} 總市電: {total_utility_power} 總利潤: {total_profit} 時段: 0")

print("\n")

for i in power_client_dict:
    profit = round(float(client_record02.get_total_quota(i)) * float(power_client_dict.get(i).get('綠電售價')), 0)
    total_quota_2 += float(client_record02.get_total_quota(i))
    total_utility_power_2 += float(client_record02.get_total_utility(i))
    total_profit_2 += profit
    print(f"電號: {client_record02.get_hd_client(i)} 轉供: {float(client_record02.get_total_quota(i))} 市電: {float(client_record02.get_total_utility(i))} 利潤: {profit} 時段: 2")
print(f"總轉供: {total_quota_2} 總市電: {total_utility_power_2} 總利潤: {total_profit_2} 時段: 2")

p = total_quota + total_quota_2
u = total_utility_power + total_utility_power_2
m = total_profit_2 + total_profit
print("\n")

for i in power_client_dict:
    t_q = float(client_record02.get_total_quota(i)) + float(client_record.get_total_quota(i))
    t_u = float(client_record02.get_total_utility(i)) + float(client_record.get_total_utility(i))
    t_p = round(float(t_q) * float(power_client_dict.get(i).get('綠電售價')), 0)
    print(f"電號: {client_record02.get_hd_client(i)} 總轉供: {t_q} 總市電: {t_u} 總利潤: {t_p} 時段: 0, 2")

print(f"總轉供: {p} 總市電: {u} 總利潤: {m} 時段: 0, 2")

電號: 64-67-0001-13-1 轉供: 0.0 市電: 379638.0 利潤: 0.0 時段: 0
電號: 64-67-0002-13-1 轉供: 455565.6 市電: 0.0 利潤: 2836124.0 時段: 0
電號: 64-67-0003-13-1 轉供: 0.0 市電: 303710.4 利潤: 0.0 時段: 0
電號: 64-67-0004-13-1 轉供: 550634.4 市電: 9715.599999999977 利潤: 3333375.0 時段: 0
電號: 64-67-0005-13-1 轉供: 0.0 市電: 2186722.5 利潤: 0.0 時段: 0
總轉供: 1006200.0 總市電: 2879786.5 總利潤: 6169499.0 時段: 0


電號: 64-67-0001-13-1 轉供: 0.0 市電: 180670.0 利潤: 0.0 時段: 2
電號: 64-67-0002-13-1 轉供: 16804.0 市電: 0.0 利潤: 104613.0 時段: 2
電號: 64-67-0003-13-1 轉供: 144536.0 市電: 0.0 利潤: 861073.0 時段: 2
電號: 64-67-0004-13-1 轉供: 481250.0 市電: 0.0 利潤: 2913343.0 時段: 2
電號: 64-67-0005-13-1 轉供: 128440.0 市電: 912243.5 利潤: 716965.0 時段: 2
總轉供: 771030.0 總市電: 1092913.5 總利潤: 4595994.0 時段: 2


電號: 64-67-0001-13-1 總轉供: 0.0 總市電: 560308.0 總利潤: 0.0 時段: 0, 2
電號: 64-67-0002-13-1 總轉供: 472369.6 總市電: 0.0 總利潤: 2940737.0 時段: 0, 2
電號: 64-67-0003-13-1 總轉供: 144536.0 總市電: 303710.4 總利潤: 861073.0 時段: 0, 2
電號: 64-67-0004-13-1 總轉供: 1031884.4 總市電: 9715.599999999977 總利潤: 6246719.0 時段: 0, 2
電號: 64-67-00

In [1167]:
total_generation = 0
total_power_t = 0
total_res = 0
total_generation_2 = 0
total_power_t_2 = 0
total_res_2 = 0
for i in power_plant_dict:
    total_power_t = float(plant_record.get_power_transfered(i))
    total_res = float(plant_record.get_residual_power(i))
    total_generation = total_power_t + total_res
    print(f"{plant_record.get_plant_name(i)} 總轉供:{plant_record.get_power_transfered(i)} 總餘電: {plant_record.get_residual_power(i)} 總產電: {total_generation} 時段: 0")
for i in power_plant_dict:
    total_power_t_2 = float(plant_record02.get_power_transfered(i))
    total_res_2 = float(plant_record02.get_residual_power(i))
    total_generation_2 = total_power_t_2 + total_res_2
    print(f"{plant_record02.get_plant_name(i)} 總轉供:{plant_record02.get_power_transfered(i)} 總餘電: {plant_record02.get_residual_power(i)}  總產電: {total_generation_2} 時段: 2")

print("\n")

for i in power_plant_dict:
    name = plant_record02.get_plant_name(i)
    total_f_r = float(plant_record02.get_residual_power(i)) + float(plant_record.get_residual_power(i))
    total_f_p = plant_record02.get_power_transfered(i) + plant_record.get_power_transfered(i)

    print(f"{plant_record02.get_plant_name(i)} 總轉供:{total_f_p} 總餘電: {total_f_r} 時段: 0,2")

電廠001 總轉供:447200.0 總餘電: 0 總產電: 447200.0 時段: 0
電廠002 總轉供:559000.0 總餘電: 0 總產電: 559000.0 時段: 0
電廠001 總轉供:342680.0 總餘電: 0  總產電: 342680.0 時段: 2
電廠002 總轉供:428350.0 總餘電: 0  總產電: 428350.0 時段: 2


電廠001 總轉供:789880.0 總餘電: 0.0 時段: 0,2
電廠002 總轉供:987350.0 總餘電: 0.0 時段: 0,2
